In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import os


In [2]:
# 1️⃣ Dataset Configuration
dataset_dir = "pest_dataset"  # Your main dataset directory
batch_size = 32
img_size = (224, 224)
split_ratio = 0.2  # 80% training, 20% validation

# 2️⃣ Data Preprocessing & Splitting
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=split_ratio
)

In [3]:
# Training Data
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset="training"
)

Found 44785 images belonging to 132 classes.


In [4]:
# Validation Data
val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset="validation"
)

Found 11129 images belonging to 132 classes.


In [5]:
num_classes = len(train_generator.class_indices)
print("Detected classes:", train_generator.class_indices)

Detected classes: {'Adristyrannus': 0, 'Aleurocanthus spiniferus': 1, 'Ampelophaga': 2, 'Aphis citricola Vander Goot': 3, 'Apolygus lucorum': 4, 'Bactrocera tsuneonis': 5, 'Beet spot flies': 6, 'Black hairy': 7, 'Brevipoalpus lewisi McGregor': 8, 'Ceroplastes rubens': 9, 'Chlumetia transversa': 10, 'Chrysomphalus aonidum': 11, 'Cicadella viridis': 12, 'Cicadellidae': 13, 'Colomerus vitis': 14, 'Dacus dorsalis(Hendel)': 15, 'Dasineura sp': 16, 'Deporaus marginatus Pascoe': 17, 'Erythroneura apicalis': 18, 'Field Cricket': 19, 'Fruit piercing moth': 20, 'Gall fly': 21, 'Icerya purchasi Maskell': 22, 'Indigo caterpillar': 23, 'Jute Stem Weevil': 24, 'Jute aphid': 25, 'Jute hairy': 26, 'Jute red mite': 27, 'Jute semilooper': 28, 'Jute stem girdler': 29, 'Jute stick insect': 30, 'Lawana imitata Melichar': 31, 'Leaf beetle': 32, 'Limacodidae': 33, 'Locust': 34, 'Locustoidea': 35, 'Lycorma delicatula': 36, 'Mango flat beak leafhopper': 37, 'Mealybug': 38, 'Miridae': 39, 'Nipaecoccus vastalor'

In [6]:
# 3️⃣ Load Pretrained ResNet50 Model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

2025-04-04 01:19:27.212218: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-04-04 01:19:27.212507: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-04-04 01:19:27.212512: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-04-04 01:19:27.212572: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-04 01:19:27.212783: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 77s 1us/step


In [7]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# 4️⃣ Add Custom Layers for Pest Classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
x = Dense(num_classes, activation="softmax")(x)  # Output layer

model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# 5️⃣ Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])



In [ ]:
model.summary()

In [10]:
# 6️⃣ Train the Model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

# 7️⃣ Save the Model
model.save("pest_detection_resnet50.h5")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


2025-04-04 01:33:00.437815: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  66/1400 ━━━━━━━━━━━━━━━━━━━━ 8:25 379ms/step - accuracy: 0.0566 - loss: 5.0275

KeyboardInterrupt: 

In [25]:
from tensorflow.keras.models import load_model

# Load the model (No need for custom objects!)
model = load_model("Models/Pest Detection ResNet50.keras")  # Update with your path

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [32]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess image
img_path = "Pest_Dataset/Ampelophaga/43864.jpg"  # Update with your image path
img = image.load_img(img_path, target_size=(224, 224))  # Resize image
img_array = image.img_to_array(img) / 255.0  # Normalize (ResNet expects values between 0 and 1)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

print("🖼️ Image preprocessed successfully!")


🖼️ Image preprocessed successfully!


In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# ✅ Step 1: Load the saved model
model_path = "Models/Pest Detection ResNet50.keras"  # Change to your path
model = tf.keras.models.load_model(model_path)

print("✅ Model Loaded Successfully!")

# ✅ Step 2: Define function to preprocess input image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Load image
    img_array = img_to_array(img) / 255.0  # Convert to array & normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# ✅ Step 3: Load class mappings (index -> label)
# If you trained using ImageDataGenerator, use this!
train_data_dir = "Pest_Dataset"  # Change to your dataset path

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1.0/255.0)
train_generator = datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=32, class_mode="categorical")

class_indices = train_generator.class_indices  # Get mapping of label -> index
index_to_class = {v: k for k, v in class_indices.items()}  # Reverse mapping

print("✅ Class mapping loaded!")

# ✅ Step 4: Predict function
def predict_pest(image_path):
    img_array = preprocess_image(image_path)
    preds = model.predict(img_array)  # Get prediction probabilities
    predicted_class = np.argmax(preds, axis=1)[0]  # Get highest prob index
    
    # Get class name
    predicted_label = index_to_class.get(predicted_class, "Unknown")

    print(f"🎯 Predicted Class: {predicted_label} (Class Index: {predicted_class})")
    print("🔢 Prediction Probabilities:", preds)

# ✅ Step 5: Run Prediction
image_path = "Pest_Dataset/Ampelophaga/43864.jpg"  # Change to your test image path
predict_pest(image_path)


✅ Model Loaded Successfully!
Found 55914 images belonging to 132 classes.
✅ Class mapping loaded!
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🎯 Predicted Class: Icerya purchasi Maskell (Class Index: 22)
🔢 Prediction Probabilities: [[0.000e+00 0.000e+00 0.000e+00 0.000e+00 5.960e-08 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 3.208e-01 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 6.279e-01 0.000e+00 3.576e-07 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.311e-06 0.000e+00 6.127e-05 0.000e+00
  1.967e-06 6.557e-07 0.000e+00 0.000e+00 0.000e+00 7.629e-06 0.000e+00
  3.207e-05 0.000e+00 0.000e+00 0.000e+00 1.293e-05 0.000e+00 7.749e-07
  1.788e-07 0.000e+00 1.490e-06 0.000e+00 0.000e+00 4.694e-02 0.000e+00
  0.000e+00 2.718e-05 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  7.749e-07 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  2.980e-07 4.295e-03 3.338e-06 1.073e-05 0.000e+00 0.000